In [ ]:
from FinMind.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
import numpy as np

# dl = DataLoader()
# df = dl.taiwan_stock_daily(stock_id='0050', start_date='2003-01-01', end_date='2023-02-25')

# df = df.rename(columns={"date": "Date"})
# df.set_index("Date" , inplace=True)
# df = df.rename(columns={"open": "Open", "max": "High", "min": "Low", "close": "Close", "Trading_Volume": "Volume"})
# df = df.rename(columns={"Trading_Volume": "Volume"})
# df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))

In [ ]:
token = pd.read_csv(r"C:\Users\Wade\Desktop\Jupyter_Python\finMindToken.txt")

In [ ]:
api = DataLoader()
api.login_by_token(api_token=token)
# api.login(user_id='user_id',password='password')
df1 = api.taiwan_futures_tick(
    futures_id='MTX',
    date='2024-06-05'
)

In [ ]:

api = DataLoader()
api.login_by_token(api_token=token)
# api.login(user_id='user_id',password='password')
df2 = api.taiwan_futures_tick(
    futures_id='MTX',
    date='2024-06-06'
)

In [ ]:
# Appending df2 to df1
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# get historical data
def get_historical_data(futures_id, date, api_token):
    from FinMind.data import DataLoader
    api = DataLoader()
    api.login_by_token(api_token=token)
    df = api.taiwan_futures_tick(futures_id=futures_id,date=date)
    return df

# date_steps
def get_dates(start_date, end_date):
    tmp_dates =[]
    delta_dates = pd.to_datetime(end_date)- pd.to_datetime(start_date)
    for i in range(delta_dates.days+1):
        tmp = pd.to_datetime(start_date)+ timedelta(days=i)
        tmp_dates.append(str(tmp.date()))
    return tmp_dates
    
# concat historical data
def concat_historical_data(futures_id, start_date, end_date,api_token):
    # The switch is to define the dataframe of the first day
    switch = True
    dates = get_dates(start_date, end_date)

    for date in dates:
        if switch:
            df = get_historical_data(futures_id, date, api_token)
            switch = False
        else:
            tmp_df = get_historical_data(futures_id, date, api_token)
            df = pd.concat([df, tmp_df], ignore_index=True)
    return df
    

In [ ]:
df = concat_historical_data('MTX','2024-06-04','2024-06-08',token)

In [ ]:
# Reshpae dataframe

MTX_month = df[df['contract_date']=='202406']

In [ ]:
MTX_month = MTX_month.reindex(np.arange(len(MTX_month)))


In [ ]:
MTX_month

In [ ]:
plt.plot(MTX_month.index.values, MTX_month.price.values)
# plt.xticks(ticks=MTX_month.index.values,labels=pd.to_datetime(MTX_month.date.values))
# plt.xlim([0,100])

In [ ]:
test = reverse_point_lower(MTX_month[['date','price']],6000,1200)
test2 = reverse_point_upper(MTX_month[['date','price']],6000,1200)

In [ ]:
test.date = pd.to_datetime(test.date)
test2.date = pd.to_datetime(test2.date)

In [ ]:

plt.figure(figsize=(15,5))
plt.scatter(MTX_month.index.values,MTX_month.price.values)
plt.plot(moving_avaerage(MTX_month.price.values,600),c='r')
plt.plot(moving_avaerage(MTX_month.price.values,9000),c='b')
plt.scatter(test.id.values,test.price.values)
plt.scatter(test2.id.values,test2.price.values)

In [ ]:
# import numpy as np
# """
# function<filter_outliers>
# I/O: 1-D array (float) /1-D array (boolean)
# """
# def filter_outliers(dataList):
#     tmpArray = np.array(dataList)
#     mean = np.nanmean(tmpArray)
#     std = np.std(tmpArray)  
#     return ((tmpArray>mean-0.5*std) & (tmpArray<mean+0.5*std))
    
# def filter_outlier_by_cluster(dataList,clusterNumber):

#     N = int(len(dataList)/clusterNumber)

#     boolean = []
#     for i in range(N):
#         tmpArray =  np.array(dataList[i*clusterNumber:(i+1)*clusterNumber])
#         mean = np.nanmean(tmpArray)
#         std = np.std(tmpArray)
#         boolean.extend(((tmpArray>mean-0.5*std) & (tmpArray<mean+0.5*std)))
        
#     tmpArray =  np.array(dataList[(i+1)*clusterNumber:])
#     mean = np.nanmean(tmpArray)
#     std = np.std(tmpArray)
#     boolean.extend(((tmpArray>mean-0.5*std) & (tmpArray<mean+0.5*std)))
#     return np.array(boolean)

# def conbined_SM(dataList,clusterNumber):
#     tmpS = filter_outlier_by_cluster(dataList,clusterNumber)
#     tmpM = filter_outliers(dataList)
#     return tmpS&tmpM

# def mutipleMethod(dataList):
#     tmp30 = filter_outlier_by_cluster(dataList,30)
#     tmp90 = filter_outlier_by_cluster(dataList,90)
#     tmp150 = filter_outlier_by_cluster(dataList,150)
#     tmpM = filter_outliers(dataList)
#     return tmp30 & tmp90 & tmp150 & tmpM
    

In [ ]:
from collections import Counter

def moving_avaerage(dataList,window):
    dataList = np.array(dataList)
    data = []
    data.extend(dataList[:window-1])
    meanWindow = np.nanmean(dataList[:window])
    data.append(meanWindow)
    for i in range(len(dataList[window:])):
        tmp = (meanWindow*window-dataList[i]+dataList[i+window])/window
        if tmp == tmp:
            meanWindow = tmp
        else: 
            meanWindow = data[-1]
        data.append(meanWindow)
    return data
    
"""
function<reverse_point>
Input: data(time,values) in pandas array + moving window range, data(time,values) in pandas array
"""
def reverse_point_lower(dataArray, window, filterNumber):
    regionMin = []
    for i in range(len(dataArray[window:])):
        minIndex = dataArray['price'][i:i+window].idxmin()
        minTimestamp = dataArray.at[minIndex, 'date']
        minWindow = dataArray.at[minIndex, 'price']
        regionMin.append(minTimestamp)
    counts = Counter(regionMin)
    elements_above_window = [element for element, count in counts.items() if count > filterNumber]

    critical_date = []
    critical_price = []
    critical_id = []
    tmp = dataArray.set_index('date')
    for d in elements_above_window:
        if len(tmp.loc[d])>1:
            critical_price.append(tmp.loc[d].iloc[0]['price'])
            critical_date.append(d)
            critical_id.append(dataArray['date'][dataArray['date']==d].index[0])
        else:
            critical_price.append(tmp.loc[d]['price'])
            critical_date.append(d)
            critical_id.append(dataArray['date'][dataArray['date']==d].index)
    dataset = {'id':critical_id,'date':critical_date,'price':critical_price}
    return pd.DataFrame(dataset)

def reverse_point_upper(dataArray,window,filterNumber):
    regionMax = []
    for i in range(len(dataArray[window:])):
        maxIndex = dataArray['price'][i:i+window].idxmax()
        maxTimestamp = dataArray.at[maxIndex, 'date']
        maxWindow = dataArray.at[maxIndex, 'price']
        regionMax.append(maxTimestamp)
    counts = Counter(regionMax)
    elements_above_window = [element for element, count in counts.items() if count > filterNumber]

    critical_date = []
    critical_price = []
    critical_id = []
    tmp = dataArray.set_index('date')
    for d in elements_above_window:
        if len(tmp.loc[d])>1:
            critical_price.append(tmp.loc[d].iloc[0]['price'])
            critical_date.append(d)
            critical_id.append(dataArray['date'][dataArray['date']==d].index[0])
        else:
            critical_price.append(tmp.loc[d]['price'])
            critical_date.append(d)
            critical_id.append(dataArray['date'][dataArray['date']==d].index)
    dataset = {'id':critical_id,'date':critical_date,'price':critical_price}
    return pd.DataFrame(dataset)

In [ ]:
test = reverse_point_lower(df[['date','price']][df['contract_date']=='202406'],6000,1200)
test2 = reverse_point_upper(df[['date','price']][df['contract_date']=='202406'],6000,1200)

In [ ]:
test = reverse_point_lower(df[['date','price']][df['contract_date']=='202406'],600,120)
test2 = reverse_point_upper(df[['date','price']][df['contract_date']=='202406'],600,120)

In [ ]:
# Get feature name
df['contract_date'].drop_duplicates()

In [ ]:
fn = '202406'
price = df['price'][df['contract_date']==fn]
volume = df['volume'][df['contract_date']==fn]
date = pd.to_datetime(df['date'][df['contract_date']==fn])
test.date = pd.to_datetime(test.date)
test2.date = pd.to_datetime(test2.date)

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(date.index.values,price)
plt.plot(date,moving_avaerage(price,6000),c='r')
plt.scatter(test.id.values,test.price.values)
plt.scatter(test2.id.values,test2.price.values)
# plt.xlim([pd.to_datetime('2024-05-21 08:40:33'),pd.to_datetime('2024-05-21 12:08:33')])
# plt.axline((0, 3), slope=1, color='C0', label='by slope')


In [ ]:
date.index.values

In [ ]:
for i in date[9900:10000]:
    print(i)

In [ ]:
import matplotlib.pyplot as plt

# 示例数据
x = [1, 2, 3, 4, 5]
y = [10, 20, 25, 30, 35]
labels = ['A', 'B', 'C', 'D', 'E']

# 创建图形和子图
fig, ax = plt.subplots()

# 绘制数据
ax.plot(x, y)

# 设置x轴刻度和标签
ax.set_xticks(x)
ax.set_xticklabels(labels)

# 添加标题和标签
ax.set_title('Example Plot')
ax.set_xlabel('Categories')
ax.set_ylabel('Values')

# 显示图形
plt.show()